In [1]:
#Importing the dependencies
import json
from openaq import OpenAQ
import pandas as pd

In [2]:
#Read the secrets file
with open(r"../secrets.json", "r") as f:
    secrets = json.load(f)

In [3]:
#Grab the 'openaq-api-key' from secrets file
openaq_key = secrets.get("openaq-api-key")

if not openaq_key:
    raise ValueError("OpenAQ API key not found in secrets.json")


In [4]:
#Connect to the OpenAQ API using the key from secrets
client = OpenAQ(api_key=openaq_key)
client

In [5]:
#Queries OpenAQ for monitoring locations around Nairobi
bbox_locations_search = client.locations.list(
    bbox=(35.782471,-1.988126,39.924316,0.834931),
    limit=1000
)
bbox_locations_search.meta

Meta(name='openaq-api', website='/', page=1, limit=1000, found=13)

In [6]:
#Actual locations data
bbox_locations_search.results

[Location(id=5994, name='Nairobi CBD', locality=None, timezone='Africa/Nairobi', country=CountryBase(id=17, code='KE', name='Kenya'), owner=OwnerBase(id=4, name='Unknown Governmental Organization'), provider=ProviderBase(id=120, name='AirNow Kenya'), is_mobile=False, is_monitor=True, instruments=[InstrumentBase(id=2, name='Government Monitor')], sensors=[SensorBase(id=16244, name='pm10 µg/m³', parameter=ParameterBase(id=1, name='pm10', units='µg/m³', display_name='PM10')), SensorBase(id=16243, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5'))], coordinates=Coordinates(latitude=-1.28156, longitude=36.81883), bounds=[36.81883, -1.28156, 36.81883, -1.28156], distance=None, datetime_first=Datetime(utc='2018-04-23T00:00:00Z', local='2018-04-23T03:00:00+03:00'), datetime_last=Datetime(utc='2018-09-13T21:00:00Z', local='2018-09-14T00:00:00+03:00')),
 Location(id=6317, name='Alliance Girls High', locality=None, timezone='Africa/Nairobi', countr

In [7]:
#Create a Clean DataFrame of Locations
locs = bbox_locations_search.results

clean_rows = []

for loc in locs:
    clean_rows.append({
        "id": loc.id,
        "name": loc.name,
        "latitude": loc.coordinates.latitude,
        "longitude": loc.coordinates.longitude,
        "provider": loc.provider.name,
        "is_monitor": loc.is_monitor,
        "sensor_count": len(loc.sensors),
        "first_seen": loc.datetime_first.local,
        "last_seen": loc.datetime_last.local
    })

df_locations = pd.DataFrame(clean_rows)
df_locations

,id,name,latitude,longitude,provider,is_monitor,sensor_count,first_seen,last_seen
0,5994,Nairobi CBD,-1.281560,36.818830,AirNow Kenya,True,2,2018-04-23T03:00:00+03:00,2018-09-14T00:00:00+03:00
1,6317,Alliance Girls High,-1.266140,36.663140,AirNow Kenya,True,2,2018-04-21T03:00:00+03:00,2018-11-21T02:00:00+03:00
2,352507,Nairobi RR,-1.234370,36.817320,AirNow,True,1,2022-06-16T20:00:00+03:00,2025-02-12T18:00:00+03:00
3,1894637,Nakuru,-0.267400,36.021800,Clarity,False,5,2023-11-28T20:03:42+03:00,2025-12-21T07:58:04+03:00
4,2156118,Nairobi,-1.331590,36.912710,Clarity,False,6,2024-01-28T18:00:32+03:00,2025-10-16T08:47:28+03:00
5,2388059,"Nyayo Embakasi, Nairobi",-1.304011,36.907681,AirGradient,False,8,2024-02-06T08:00:00+03:00,2024-05-09T21:00:00+03:00
6,3394396,Nairobi CMR,-1.232511,36.817320,AirNow,True,1,2025-02-06T21:00:00+03:00,2025-06-26T13:00:00+03:00
7,3394397,Nairobi CMR,-1.232511,36.817320,AirNow,True,1,2025-02-11T01:00:00+03:00,2025-02-12T00:00:00+03:00
8,3399524,Nairobi RR,-1.222910,36.805490,AirNow,True,1,2025-02-12T16:00:00+03:00,2025-02-17T19:00:00+03:00
9,5119477,Kitisuru,-1.225252,36.758078,AirGradient,False,5,2025-07-18T16:00:00+03:00,2025-12-09T12:00:00+03:00


In [8]:
#Sort all locations by their most recent reading timestamp
df_locations['first_seen'] = pd.to_datetime(df_locations['first_seen'])
df_locations['last_seen'] = pd.to_datetime(df_locations['last_seen'])
df_locations = df_locations.sort_values(by='last_seen', ascending=False)
df_locations


,id,name,latitude,longitude,provider,is_monitor,sensor_count,first_seen,last_seen
10,5199863,Kihumo Village || Antenna Array,-1.261836,36.686049,AirGradient,False,5,2025-07-25 14:00:00+03:00,2025-12-21 08:00:00+03:00
3,1894637,Nakuru,-0.267400,36.021800,Clarity,False,5,2023-11-28 20:03:42+03:00,2025-12-21 07:58:04+03:00
9,5119477,Kitisuru,-1.225252,36.758078,AirGradient,False,5,2025-07-18 16:00:00+03:00,2025-12-09 12:00:00+03:00
4,2156118,Nairobi,-1.331590,36.912710,Clarity,False,6,2024-01-28 18:00:32+03:00,2025-10-16 08:47:28+03:00
6,3394396,Nairobi CMR,-1.232511,36.817320,AirNow,True,1,2025-02-06 21:00:00+03:00,2025-06-26 13:00:00+03:00
11,5672958,KENBIFS01,-1.281800,36.823200,Clean Air Catalyst,True,8,2024-09-03 17:00:00+03:00,2025-05-31 18:00:00+03:00
12,5672959,KENBIMLK01,-1.274200,36.899000,Clean Air Catalyst,True,11,2024-09-03 17:00:00+03:00,2025-05-31 18:00:00+03:00
8,3399524,Nairobi RR,-1.222910,36.805490,AirNow,True,1,2025-02-12 16:00:00+03:00,2025-02-17 19:00:00+03:00
2,352507,Nairobi RR,-1.234370,36.817320,AirNow,True,1,2022-06-16 20:00:00+03:00,2025-02-12 18:00:00+03:00
7,3394397,Nairobi CMR,-1.232511,36.817320,AirNow,True,1,2025-02-11 01:00:00+03:00,2025-02-12 00:00:00+03:00


In [10]:
#Create a dictionary for mapping each location ID to its location name.
locations_info = {}

for location in bbox_locations_search.results:
    locations_info[location.id] = location.name

locations_info

{5994: 'Nairobi CBD',
 6317: 'Alliance Girls High',
 352507: 'Nairobi RR',
 1894637: 'Nakuru',
 2156118: 'Nairobi',
 2388059: 'Nyayo Embakasi, Nairobi',
 3394396: 'Nairobi CMR',
 3394397: 'Nairobi CMR',
 3399524: 'Nairobi RR',
 5119477: 'Kitisuru',
 5199863: 'Kihumo Village || Antenna Array',
 5672958: 'KENBIFS01',
 5672959: 'KENBIMLK01'}

In [11]:
#Look at location 'nakuru' data
bbox_locations_search.results[3].__dict__

{'id': 1894637,
 'name': 'Nakuru',
 'locality': None,
 'timezone': 'Africa/Nairobi',
 'country': CountryBase(id=17, code='KE', name='Kenya'),
 'owner': OwnerBase(id=9, name='Clarity'),
 'provider': ProviderBase(id=166, name='Clarity'),
 'is_mobile': False,
 'is_monitor': False,
 'instruments': [InstrumentBase(id=4, name='Clarity Sensor')],
 'sensors': [SensorBase(id=7466383, name='pm1 µg/m³', parameter=ParameterBase(id=19, name='pm1', units='µg/m³', display_name='PM1')),
  SensorBase(id=7466384, name='pm10 µg/m³', parameter=ParameterBase(id=1, name='pm10', units='µg/m³', display_name='PM10')),
  SensorBase(id=7466385, name='pm25 µg/m³', parameter=ParameterBase(id=2, name='pm25', units='µg/m³', display_name='PM2.5')),
  SensorBase(id=7466386, name='temperature c', parameter=ParameterBase(id=100, name='temperature', units='c', display_name='Temperature (C)')),
  SensorBase(id=7466387, name='temperature f', parameter=ParameterBase(id=128, name='temperature', units='f', display_name='Tempe

In [12]:
#Save locations_info as a JSON file
with open(r"..\locations.json", "w") as f:
    json.dump(locations_info, f, indent=4, sort_keys=True)
    